In [2]:
%load_ext autoreload
%autoreload 2

try:
    from google.colab import drive
    drive.mount('/content/drive')
    import os
    GOOGLE_DRIVE_PATH_AFTER_MYDRIVE = 'Vgg16'
    GOOGLE_DRIVE_PATH = os.path.join('drive', 'My Drive', GOOGLE_DRIVE_PATH_AFTER_MYDRIVE)
    print(os.listdir(GOOGLE_DRIVE_PATH))
    import sys
    sys.path.append(GOOGLE_DRIVE_PATH)
except:
    pass

import time, os

import torch
from torch import Tensor
import torchvision
import matplotlib.pyplot as plt
import statistics
import random
import time
import math
import numpy as np
%matplotlib inline
plt.rcParams['figure.figsize'] = (6, 4)
plt.rcParams['font.size'] = 10
import toolset as ts
from toolset import *
from toolset.utils import *
from toolset.data import *
from toolset.helper import *
from toolset.solver import *
data_dict = ts.data.preprocess_cifar10(cuda=True, dtype=torch.float64, flatten=False,show_examples=False)
print('Train data shape: ', data_dict['X_train'].shape)
print('Train labels shape: ', data_dict['y_train'].shape)
print('Validation data shape: ', data_dict['X_val'].shape)
print('Validation labels shape: ', data_dict['y_val'].shape)
print('Test data shape: ', data_dict['X_test'].shape)
print('Test labels shape: ', data_dict['y_test'].shape)
from fully_connected_networks import *
from convolutional_networks import *
import gc




# half = {
#   'X_train': data_dict['X_train'][:data_dict['X_train'].shape[0]//2],
#   'y_train': data_dict['y_train'][:data_dict['X_train'].shape[0]//2],
#   'X_val': data_dict['X_val'][:data_dict['X_val'].shape[0]//2],
#   'y_val': data_dict['y_val'][:data_dict['X_val'].shape[0]//2],
# }

input_dims = data_dict['X_train'].shape[1:]

if torch.cuda.is_available:
    print('Good to go!')
else:
    print('Please set GPU via Edit -> Notebook Settings.')
    
def clear(x=None):
    x = None
    gc.collect(),torch.cuda.empty_cache()
    # 不会影响当前有效（非None）的变量。
    # gc.collect() 会清理掉Python环境中所有未被引用的对象，而 torch.cuda.empty_cache() 会清理掉PyTorch在CUDA中的缓存
clear()



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Train data shape:  torch.Size([40000, 3, 32, 32])
Train labels shape:  torch.Size([40000])
Validation data shape:  torch.Size([10000, 3, 32, 32])
Validation labels shape:  torch.Size([10000])
Test data shape:  torch.Size([10000, 3, 32, 32])
Test labels shape:  torch.Size([10000])
Good to go!


In [1]:

def search_params():
  learing_rate = [5e-2, 2.5e-2, 1e-3, 7.5e-4, 5e-4]
  regs = [1e-5, 5e-4]
  small_data = {
  'X_train': data_dict['X_train'][:350],
  'y_train': data_dict['y_train'][:350],
  'X_val': data_dict['X_val'][:1],
  'y_val': data_dict['y_val'][:1],
  }
  for reg in regs:
    for lr in learing_rate:
        net16 = VggNet(
                        input_dims=(3, 32, 32),
                        num_filters=(64, 64, 
                                    128, 128, 
                                    256, 256, 256, 
                                    512, 512, 512, 
                                    512, 512, 512),
                        num_FC=(4096, 4096, 10),
                        max_pools=(1, 3, 6, 9, 12),
                        weight_scale='kaiming',
                        reg=reg,
                        dtype=torch.float32,
                        # print_params=False,
                        dropout=0.5,
                        device='cuda'
        )
        # net16.check_loss(data_dict=data_dict)

        solver = Solver(net16, small_data,
                        num_epochs=100, batch_size=100,
                        optim_config={
                        'learning_rate': lr,
                        },
                        update_rule=adam,
                        verbose=True, 
                        device='cuda',
                        lr_decay=0.95,
                        print_every=50,
                        check_batch_size=500
                        )
        solver.train(time_limit=4200)
        print(f"使用学习率{lr}")
        plot_solver(solver)
        solver._reset()
        clear(solver)
        clear(net16)
# lr 5e-4, reg 5e-4

In [3]:
clear()
net_mini = VggNet(
                input_dims=(3, 32, 32),
                num_filters=(64, 
                            128, 128 ,
                            256, 256),
                num_FC=(128, 10),
                max_pools=(0, 2, 4),
                weight_scale='kaiming',
                reg=5e-4,
                kaiming_ratio=0.5,
                dtype=torch.float32,
                print_params=True,
                dropout=0.5,
                device='cuda'
)
net_mini.check_loss(data_dict=data_dict)
solver = Solver(net_mini, data_dict,
                num_epochs=15, batch_size=100,
                optim_config={
                'learning_rate': 5e-4,
                },
                update_rule=adam,
                verbose=True, 
                device='cuda',
                lr_decay=0.95,
                print_every=50,
                check_batch_size=250
                )


使用dropout。
参数:
	W1: torch.Size([64, 3, 3, 3])
	b1: torch.Size([64])
	gamma1: torch.Size([64])
	beta1: torch.Size([64])
	W2: torch.Size([128, 64, 3, 3])
	b2: torch.Size([128])
	gamma2: torch.Size([128])
	beta2: torch.Size([128])
	W3: torch.Size([128, 128, 3, 3])
	b3: torch.Size([128])
	gamma3: torch.Size([128])
	beta3: torch.Size([128])
	W4: torch.Size([256, 128, 3, 3])
	b4: torch.Size([256])
	gamma4: torch.Size([256])
	beta4: torch.Size([256])
	W5: torch.Size([256, 256, 3, 3])
	b5: torch.Size([256])
	gamma5: torch.Size([256])
	beta5: torch.Size([256])
	W6: torch.Size([4096, 128])
	b6: torch.Size([128])
	gamma6: torch.Size([128])
	beta6: torch.Size([128])
	W7: torch.Size([128, 10])
	b7: torch.Size([10])
tensor([[-8.9670e+11,  2.3870e+11, -4.2219e+11,  8.3256e+11,  7.5531e+11,
          3.0886e+12, -7.9234e+11, -1.2743e+12,  1.3023e+12, -2.8150e+12],
        [-1.2555e+12,  1.4896e+12,  5.8005e+11,  1.4166e+12,  8.0663e+11,
          3.2067e+12,  2.0820e+11, -1.8838e+12,  1.6460e+12, -2.7

In [4]:
clear()
solver.train(time_limit=1200)


(用时 0.06s; 迭代次数1 / 6000) loss: 2.581132
(Epoch 0 / 15, train 准确率: 11.40%, val准确率: 11.99%
更新:当前准确率11.99, 大于之前的最佳准确率0.00

复制BN参数成功
复制BN参数成功
复制BN参数成功
复制BN参数成功
复制BN参数成功
复制BN参数成功
复制BN参数成功
复制BN参数成功
复制BN参数成功
复制BN参数成功
复制BN参数成功
复制BN参数成功
复制BN参数成功
复制BN参数成功
复制BN参数成功
复制BN参数成功
复制BN参数成功
复制BN参数成功
复制BN参数成功
复制BN参数成功
复制BN参数成功
复制BN参数成功
复制BN参数成功
复制BN参数成功
复制BN参数成功
复制BN参数成功
(用时 2.94s; 迭代次数51 / 6000) loss: 1.925211
(用时 5.05s; 迭代次数101 / 6000) loss: 1.621663
(用时 7.26s; 迭代次数151 / 6000) loss: 1.641162
(用时 9.41s; 迭代次数201 / 6000) loss: 1.357378
(用时 11.54s; 迭代次数251 / 6000) loss: 1.475959
(用时 13.68s; 迭代次数301 / 6000) loss: 1.163408
(用时 15.79s; 迭代次数351 / 6000) loss: 1.253652
(Epoch 1 / 15, train 准确率: 59.70%, val准确率: 58.92%
更新:当前准确率58.92, 大于之前的最佳准确率11.99

复制BN参数成功
复制BN参数成功
复制BN参数成功
复制BN参数成功
复制BN参数成功
复制BN参数成功
复制BN参数成功
复制BN参数成功
复制BN参数成功
复制BN参数成功
复制BN参数成功
复制BN参数成功
复制BN参数成功
复制BN参数成功
复制BN参数成功
复制BN参数成功
复制BN参数成功
复制BN参数成功
复制BN参数成功
复制BN参数成功
复制BN参数成功
复制BN参数成功
复制BN参数成功
复制BN参数成功
复制BN参数成功
复制BN参数成功
(用时 18.69s; 迭代次数401 / 6000) loss: 1

In [6]:
acc = solver.check_accuracy(data_dict['X_val'],data_dict['y_val'])
print(acc)

0.8130999803543091


In [ ]:
plot_solver(solver)

solver.checkpoint_name = "5_25_mini"
solver._save_checkpoint()
sv_path = "./drive/My Drive/Vgg16/net16_5_25_mini.pth"
net_mini.save(sv_path)

In [4]:

net_mini_test = VggNet()
net_mini_test.load("./net16_5_25_mini.pth",dtype=torch.float32,device='cuda')

solver_checker = Solver(net_mini_test, data_dict,device='cuda')
acc = solver_checker.check_accuracy(data_dict['X_val'],data_dict['y_val'])
print(acc)


成功加载checkpoint文件: ./net16_5_25_mini.pth
0.8155999779701233


In [6]:
net_mini_test.print_params()

参数:
	W1: torch.Size([64, 3, 3, 3])
	b1: torch.Size([64])
	gamma1: torch.Size([64])
	beta1: torch.Size([64])
	W2: torch.Size([128, 64, 3, 3])
	b2: torch.Size([128])
	gamma2: torch.Size([128])
	beta2: torch.Size([128])
	W3: torch.Size([128, 128, 3, 3])
	b3: torch.Size([128])
	gamma3: torch.Size([128])
	beta3: torch.Size([128])
	W4: torch.Size([256, 128, 3, 3])
	b4: torch.Size([256])
	gamma4: torch.Size([256])
	beta4: torch.Size([256])
	W5: torch.Size([256, 256, 3, 3])
	b5: torch.Size([256])
	gamma5: torch.Size([256])
	beta5: torch.Size([256])
	W6: torch.Size([4096, 128])
	b6: torch.Size([128])
	gamma6: torch.Size([128])
	beta6: torch.Size([128])
	W7: torch.Size([128, 10])
	b7: torch.Size([10])


In [2]:
net16 = VggNet(
                input_dims=(3, 32, 32),
                num_filters=(64, 64, 
                            128, 128, 
                            256, 256, 256, 
                            512, 512, 512, 
                            512, 512, 512),
                num_FC=(4096, 4096, 10),
                max_pools=(1, 3, 6, 9, 12),
                weight_scale='kaiming',
                kaiming_ratio=0.1,
                reg=5e-4,
                dtype=torch.float32,
                print_params=True,
                dropout=0.5,
                device='cuda'
)
net16.check_loss(data_dict=data_dict)
solver = Solver(net16, data_dict,
                num_epochs=10, batch_size=100,
                optim_config={
                'learning_rate': 1e-4,
                },
                update_rule=adam,
                verbose=True, 
                device='cuda',
                lr_decay=0.95,
                print_every=50,
                check_batch_size=500
                )


使用dropout。
参数:
	W1: torch.Size([64, 3, 3, 3])
	b1: torch.Size([64])
	gamma1: torch.Size([64])
	beta1: torch.Size([64])
	W2: torch.Size([64, 64, 3, 3])
	b2: torch.Size([64])
	gamma2: torch.Size([64])
	beta2: torch.Size([64])
	W3: torch.Size([128, 64, 3, 3])
	b3: torch.Size([128])
	gamma3: torch.Size([128])
	beta3: torch.Size([128])
	W4: torch.Size([128, 128, 3, 3])
	b4: torch.Size([128])
	gamma4: torch.Size([128])
	beta4: torch.Size([128])
	W5: torch.Size([256, 128, 3, 3])
	b5: torch.Size([256])
	gamma5: torch.Size([256])
	beta5: torch.Size([256])
	W6: torch.Size([256, 256, 3, 3])
	b6: torch.Size([256])
	gamma6: torch.Size([256])
	beta6: torch.Size([256])
	W7: torch.Size([256, 256, 3, 3])
	b7: torch.Size([256])
	gamma7: torch.Size([256])
	beta7: torch.Size([256])
	W8: torch.Size([512, 256, 3, 3])
	b8: torch.Size([512])
	gamma8: torch.Size([512])
	beta8: torch.Size([512])
	W9: torch.Size([512, 512, 3, 3])
	b9: torch.Size([512])
	gamma9: torch.Size([512])
	beta9: torch.Size([512])
	W10: t

In [ ]:
solver.train(time_limit=7200)
plot_solver(solver)
acc = solver.check_accuracy(data_dict['X_val'],data_dict['y_val'])
print(acc)
# solver.checkpoint_name = "5_25"
# solver._save_checkpoint()
# sv_path = "net16_5_25.pth"
# net16.save(sv_path)

In [ ]:
net16_check = VggNet()
net16_check.load(path=sv_path,dtype=torch.float32,device='cuda')
solver_checker = Solver(net16_check, data_dict,optim_config={
                'learning_rate': 0.001,
                },device='cuda')
acc = solver_checker.check_accuracy(data_dict['X_val'],data_dict['y_val'])
print(acc)


In [ ]:
X_samples = data_dict['X_val'][114:124]
y_samples = data_dict['y_val'][114:124]
X_samples.to('cuda')
y_samples.to('cuda')


In [ ]:
classes = ["plane", "car", "bird", "cat", "deer", "dog", "frog", "horse", "ship", "truck"]

forecast = []
for i in net16.loss(X_samples).argmax(dim=1):
    forecast.append(classes[i])

In [ ]:
tensor_to_imggrid_show(X_samples)
print(forecast)

In [ ]:
solver.check_accuracy(data_dict['X_train'], data_dict['y_train'])

In [ ]:
loss, _ = net16.loss(data_dict['X_train'][:10], data_dict['y_train'][:10])
print(loss)

In [ ]:
for i in range(50):
    print(solver.check_accuracy(data_dict['X_train'], data_dict['y_train'],100))

In [ ]:
#!!!!!
clear()
if 0:
    solver._reset()
    clear(solver)

In [ ]:
# from fully_connected_networks import sgd_momentum
# model = DeepConvNet(input_dims=input_dims, num_classes=10,
#                     num_filters=[8, 8, 8],
#                     max_pools=[0, 1, 2],
#                     weight_scale='kaiming',
#                     batchnorm=False,
#                     reg=1e-5, dtype=torch.float32, device='cuda')
# solver = Solver(model, small_data,
#             num_epochs=1000, batch_size=50,
#             update_rule = adam,
#             optim_config={
#             'learning_rate': 0.001,
#             },
#             verbose=True, device='cuda',print_every=2000)
# solver.train(time_limit=60)

In [ ]:
from toolset.helper import plot_solver
plot_solver(solver)


In [ ]:

vgg16_model.save("D:/Paper/Vgg16/60.pth")

In [ ]:
# 用于google
PATH = './drive/My Drive/tran.pth'
vgg16_model.save(PATH)


In [ ]:


solver = Solver(vgg16_model, data_dict,
                num_epochs=10, batch_size=50,
                optim_config={
                'learning_rate': 5e-5,
                },
                update_rule=adam,
                verbose=True, device='cuda',lr_decay=0.95,print_every=50)

In [ ]:
solver.train(time_limit=300)